In [1]:
search_host = 'scsearch.search.windows.net'
admin_key = 'BnuFulcj1IJrGfWDwTYOo5HmkTskjvQe6RU1e2qSunAzSeDnxXgf'
query_key = '3IXZV1z2f30JPsQcuF0fnT1I0XXKsYId9z1EI9PCe3AzSeDzipuA'
blob_connection_string = 'DefaultEndpointsProtocol=https;AccountName=sharecaredatalake;AccountKey=W+tfmRuw8G4bEQeAA/6YzSd3jsEDq1ub2LtVnvaRmD/A7ZiRXqcdnUI197qDIieR3IKCXDB7bM58+AStHlIvww==;EndpointSuffix=core.windows.net'
cgsvc_key = 'e2d4bf9615914b10a45cc3e94ffc3207'
cgsvc_endpoint = 'https://sharecare-cogsvc.cognitiveservices.azure.com/'
azure_function = 'https://totipufrequentwords.azurewebsites.net/api/FrequentWords'

## Main REST Function

In [2]:
def CallAzureSearch (method="GET", function="", body=None):
    import http.client, json, os, sys, urllib
    from urllib.parse import urlparse

    headers = {
        'Content-Type': 'application/json',
        'api-key': admin_key
    }

    # Request parameters
    params = urllib.parse.urlencode({
        'api-version':'2020-06-30'
    })

    request_path = "/{0}?{1}".format(function, params)
    
    try:
        conn = http.client.HTTPSConnection(search_host)    
        conn.request (method, request_path, json.dumps(body), headers)
        response = conn.getresponse ()
        print(response.status)
        data = response.read().decode("UTF-8")
        result = None
        if len(data) > 0:
            result = json.loads(data)
        return result
    except Exception as ex:
        raise ex

## Create a Data Source

In [3]:
import json

body = {
        "name": "sharecare-documents",
        "type": "azureblob",
        "credentials" : { "connectionString" : blob_connection_string },
        "container" : { "name" : "documents" }
    }

result = CallAzureSearch(method="POST", function="datasources", body = body)
if (result != None):
    print(json.dumps(result, sort_keys=True, indent=2))

201
{
  "@odata.context": "https://scsearch.search.windows.net/$metadata#datasources/$entity",
  "@odata.etag": "\"0x8DACB4381696C29\"",
  "container": {
    "name": "documents",
    "query": null
  },
  "credentials": {
    "connectionString": null
  },
  "dataChangeDetectionPolicy": null,
  "dataDeletionDetectionPolicy": null,
  "description": null,
  "encryptionKey": null,
  "name": "sharecare-documents",
  "subtype": null,
  "type": "azureblob"
}


## Verify a Data Source exists

In [4]:
import json

result = CallAzureSearch(function="datasources")
if (len(result) > 0):
    print(json.dumps(result, sort_keys=True, indent=2))

200
{
  "@odata.context": "https://scsearch.search.windows.net/$metadata#datasources",
  "value": [
    {
      "@odata.etag": "\"0x8DACB4381696C29\"",
      "container": {
        "name": "documents",
        "query": null
      },
      "credentials": {
        "connectionString": null
      },
      "dataChangeDetectionPolicy": null,
      "dataDeletionDetectionPolicy": null,
      "description": null,
      "encryptionKey": null,
      "name": "sharecare-documents",
      "subtype": null,
      "type": "azureblob"
    }
  ]
}


## Add a SynonymMap

In [5]:
import json

body = {
    "name": "locationsynonyms",
    "format": "solr",
    "synonyms": "USA, United States, America, United States of America\n" \
        "UK, United Kingdom, Britain, Great Britain\n" \
        "UAE, United Arab Emirates, Emirates\n"
}

result = CallAzureSearch(method="POST", function="synonymmaps", body = body)
if (len(result) > 0):
    print(json.dumps(result, sort_keys=True, indent=2))

201
{
  "@odata.context": "https://scsearch.search.windows.net/$metadata#synonymmaps/$entity",
  "@odata.etag": "\"0x8DACB438F04B182\"",
  "encryptionKey": null,
  "format": "solr",
  "name": "locationsynonyms",
  "synonyms": "USA, United States, America, United States of America\nUK, United Kingdom, Britain, Great Britain\nUAE, United Arab Emirates, Emirates\n"
}


## Update an Index

### Delete Existing Index

In [ ]:
result = CallAzureSearch(method="DELETE", function="indexes/margies-docs-index", body = body)
if (result != None):
    print(json.dumps(result, sort_keys=True, indent=2))

### Create New Index

In [6]:
body = {    
    "fields": [
        { "name": "id", "type": "Edm.String", "key": True, "searchable": True, "filterable": True, "facetable": False, "sortable": True },
        { "name": "url", "type": "Edm.String", "searchable": True, "filterable": True, "facetable": False, "sortable": True },
        { "name": "file_name", "type": "Edm.String", "searchable": True, "filterable": True, "facetable": False, "sortable": True },
        { 
            "name": "content", 
            "type": "Edm.String", 
            "searchable": True, 
            "filterable": True, 
            "facetable": False, 
            "sortable": False, 
            "synonymMaps" : [ "locationsynonyms" ]
        },
        { "name": "size", "type": "Edm.Int64", "searchable": False, "filterable": True, "facetable": False, "sortable": True },
        { "name": "last_modified", "type": "Edm.DateTimeOffset", "searchable": False, "filterable": True, "facetable": False, "sortable": True },
        { "name": "sentiment_score", "type": "Edm.Double", "searchable": False, "filterable": True, "facetable": False, "sortable": True },
        { "name": "key_phrases", "type": "Collection(Edm.String)", "searchable": True, "filterable": False, "facetable": False, "sortable": False, "synonymMaps" : [ "locationsynonyms" ] },
        { "name": "people", "type": "Collection(Edm.String)", "searchable": True, "filterable": True, "facetable": False, "sortable": False },
        { "name": "locations", "type": "Collection(Edm.String)", "searchable": True, "filterable": True, "facetable": False, "sortable": False, "synonymMaps" : [ "locationsynonyms" ] },
        { "name": "links", "type": "Collection(Edm.String)", "searchable": True, "filterable": True, "facetable": False, "sortable": False },
        { "name": "entities", "type": "Collection(Edm.String)", "searchable": True, "filterable": True, "facetable": False, "sortable": False },
        { "name": "image_descriptions", "type": "Collection(Edm.String)", "searchable": True, "filterable": True, "facetable": False, "sortable": False, "synonymMaps" : [ "locationsynonyms" ] },
        { "name": "image_text", "type": "Collection(Edm.String)", "searchable": True, "filterable": True, "facetable": False, "sortable": False, "synonymMaps" : [ "locationsynonyms" ] },
        { "name": "merged_text", "type": "Edm.String", "searchable": True, "filterable": False, "facetable": False, "sortable": False, "synonymMaps" : [ "locationsynonyms" ] },
        { "name": "top_words", "type": "Collection(Edm.String)", "searchable": True, "filterable": True, "facetable": False, "sortable": False, "synonymMaps" : [ "locationsynonyms" ] },

    ],
    "suggesters": [
        {
            "name": "location_suggester",
            "searchMode": "analyzingInfixMatching",
            "sourceFields": ["locations"]
        }
    ],
    "scoringProfiles" : [
        {
            "name": "boostRelevance",
            "text": {
                "weights" : {
                    "key_phrases": 4.0
                }
            },
            "functions": [
                {
                    "type": "magnitude",
                    "fieldName": "size",
                    "interpolation": "linear",                    
                    "boost": 2,
                    "magnitude": {
                        "boostingRangeStart": 1000,
                        "boostingRangeEnd": 500000,
                        "constantBoostBeyondRange": True
                    }
                },
                {
                    "type": "freshness",
                    "fieldName": "last_modified",
                    "boost": 2,
                    "interpolation": "quadratic",
                    "freshness": {
                        "boostingDuration": "P365D"
                    }
                }
            ]

        }
    ]
}

result = CallAzureSearch(method="PUT", function="indexes/sharecare-docs-index", body = body)
if (result != None):
    print(json.dumps(result, sort_keys=True, indent=2))

201
{
  "@odata.context": "https://scsearch.search.windows.net/$metadata#indexes/$entity",
  "@odata.etag": "\"0x8DACB43B48ADF68\"",
  "analyzers": [],
  "charFilters": [],
  "corsOptions": null,
  "defaultScoringProfile": null,
  "encryptionKey": null,
  "fields": [
    {
      "analyzer": null,
      "facetable": false,
      "filterable": true,
      "indexAnalyzer": null,
      "key": true,
      "name": "id",
      "retrievable": true,
      "searchAnalyzer": null,
      "searchable": true,
      "sortable": true,
      "synonymMaps": [],
      "type": "Edm.String"
    },
    {
      "analyzer": null,
      "facetable": false,
      "filterable": true,
      "indexAnalyzer": null,
      "key": false,
      "name": "url",
      "retrievable": true,
      "searchAnalyzer": null,
      "searchable": true,
      "sortable": true,
      "synonymMaps": [],
      "type": "Edm.String"
    },
    {
      "analyzer": null,
      "facetable": false,
      "filterable": true,
      "indexAnal

## Create a Skillset

In [8]:
body = {
    "description": "Detects sentiment, extracts key-phrases, extracts entities",
    "skills":
    [
        {
            "@odata.type": "#Microsoft.Skills.Text.SentimentSkill",            
            "inputs": [
                { "name": "text", "source": "/document/content" }                
            ],
            "outputs": [
                { "name": "score", "targetName": "sentiment_score" }
            ]
        },
        {
            "@odata.type": "#Microsoft.Skills.Text.KeyPhraseExtractionSkill",            
            "inputs": [
                { "name": "text", "source": "/document/content" }                
            ],
            "outputs": [
                { "name": "keyPhrases", "targetName": "key_phrases" }
            ]
        },
        {
            "@odata.type": "#Microsoft.Skills.Text.EntityRecognitionSkill",
            "categories": [ "Organization", "Person", "Location", "URL", "DateTime" ],
            "defaultLanguageCode": "en",
            "inputs": [
                { "name": "text", "source": "/document/content" }
            ],
            "outputs": [
                { "name": "persons", "targetName": "people" },
                { "name": "locations", "targetName": "locations" },
                { "name": "urls", "targetName": "links" },
                { "name": "entities", "targetName": "entities" }
            ]
        }
    ],
    "cognitiveServices": 
      {
        "@odata.type": "#Microsoft.Azure.Search.CognitiveServicesByKey",
        "description": "Margies Cognitive Services",
        "key": cgsvc_key
      },
}

result = CallAzureSearch(method="PUT", function="skillsets/sharecare-docs-skillset", body = body)
if (result != None):
    print(json.dumps(result, sort_keys=True, indent=2))

201
{
  "@odata.context": "https://scsearch.search.windows.net/$metadata#skillsets/$entity",
  "@odata.etag": "\"0x8DACB456E112743\"",
  "cognitiveServices": {
    "@odata.type": "#Microsoft.Azure.Search.CognitiveServicesByKey",
    "description": "Margies Cognitive Services",
    "key": "e2d4bf9615914b10a45cc3e94ffc3207"
  },
  "description": "Detects sentiment, extracts key-phrases, extracts entities",
  "encryptionKey": null,
  "knowledgeStore": null,
  "name": "sharecare-docs-skillset",
  "skills": [
    {
      "@odata.type": "#Microsoft.Skills.Text.SentimentSkill",
      "context": null,
      "defaultLanguageCode": null,
      "description": null,
      "inputs": [
        {
          "inputs": [],
          "name": "text",
          "source": "/document/content",
          "sourceContext": null
        }
      ],
      "name": null,
      "outputs": [
        {
          "name": "score",
          "targetName": "sentiment_score"
        }
      ]
    },
    {
      "@odata.type

## Update an indexer

In [10]:
import json

body = {
    "name" : "sharecare-docs-indexer",
    "dataSourceName" : "sharecare-documents",
    "targetIndexName" : "sharecare-docs-index",
    "skillsetName": "sharecare-docs-skillset",
    "fieldMappings" : [
        { "sourceFieldName" : "metadata_storage_name", "targetFieldName" : "id", "mappingFunction" : { "name" : "base64Encode" } },
        { "sourceFieldName" : "metadata_storage_path", "targetFieldName" : "url" },
        { "sourceFieldName" : "metadata_storage_name", "targetFieldName" : "file_name" },
        { "sourceFieldName" : "content", "targetFieldName" : "content" },
        { "sourceFieldName" : "metadata_storage_size", "targetFieldName" : "size" },
        { "sourceFieldName" : "metadata_storage_last_modified", "targetFieldName" : "last_modified" }
    ],
    "outputFieldMappings" : [
        { "sourceFieldName" : "/document/sentiment_score", "targetFieldName" : "sentiment_score" },
        { "sourceFieldName" : "/document/key_phrases/*", "targetFieldName" : "key_phrases" },
        { "sourceFieldName" : "/document/people", "targetFieldName" : "people" },
        { "sourceFieldName" : "/document/locations", "targetFieldName" : "locations" },
        { "sourceFieldName" : "/document/links", "targetFieldName" : "links" },
        { "sourceFieldName" : "/document/entities", "targetFieldName" : "entities" },
    ],
    "parameters": { 
        "maxFailedItems":-1,
        "maxFailedItemsPerBatch":-1,
        "configuration" : {"dataToExtract":"contentAndMetadata"}
        }
}

result = CallAzureSearch(method="PUT", function="indexers/sharecare-docs-indexer", body = body)
if (result != None):
    print(json.dumps(result, sort_keys=True, indent=2))

204


## Create an indexer

In [ ]:
import json

body = {
    "name" : "sharecare-docs-indexer",
    "dataSourceName" : "sharecare-documents",
    "targetIndexName" : "sharecare-docs-index",
    "fieldMappings" : [
        { "sourceFieldName" : "metadata_storage_name", "targetFieldName" : "id", "mappingFunction" : { "name" : "base64Encode" } },
        { "sourceFieldName" : "metadata_storage_path", "targetFieldName" : "url" },
        { "sourceFieldName" : "metadata_storage_name", "targetFieldName" : "file_name" },
        { "sourceFieldName" : "content", "targetFieldName" : "content" },
        { "sourceFieldName" : "metadata_storage_size", "targetFieldName" : "size" },
        { "sourceFieldName" : "metadata_storage_last_modified", "targetFieldName" : "last_modified" }
      ],
    "parameters": { 
        "maxFailedItems":-1,
        "maxFailedItemsPerBatch":-1,
        "configuration" : {"dataToExtract":"contentAndMetadata"}
        }
}

result = CallAzureSearch(method="POST", function="indexers", body = body)
if (result != None):
    print(json.dumps(result, sort_keys=True, indent=2))

## Reset and rerun the indexer

In [11]:
# Reset the indexer
result = CallAzureSearch(method="POST", function="indexers/sharecare-docs-indexer/reset", body = body)
if (result != None):
    print(json.dumps(result, sort_keys=True, indent=2))

# Run the indexer
result = CallAzureSearch(method="POST", function="indexers/sharecare-docs-indexer/run", body = body)
if (result != None):
    print(json.dumps(result, sort_keys=True, indent=2))


204
202


## Search Functions

In [ ]:
def QuerySearch (index, params):
    import http.client, json, os, sys, urllib
    from urllib.parse import urlparse

    headers = {
        'Content-Type': 'application/json',
        'api-key': query_key
    }

    request_path = "/indexes/{0}/docs?{1}".format(index, params)
    
    try:
        conn = http.client.HTTPSConnection(search_host)    
        conn.request ("GET", request_path, None, headers)
        response = conn.getresponse ()
        print(response.status)
        data = response.read().decode("UTF-8")
        result = json.loads(data)
        return result

    except Exception as ex:
        raise ex

print('Ready to accept search queries...')

## Search based on synonyms

In [ ]:
import urllib

# Request parameters
params = urllib.parse.urlencode({
    'search': 'Britain',
    'queryType': 'simple',
    'searchMode': 'all',
    '$count': 'true',
    '$select':'file_name, url',    
    'api-version':'2020-06-30'
})

result = QuerySearch('margies-docs-index', params)
print("Count: " + str(result["@odata.count"]))

for val in result["value"]:    
    print("\n", val["url"])
    print(val["file_name"])

## Suggestions

### Query

In [ ]:
def SuggestionQuery (index, params):
    import http.client, json, os, sys, urllib
    from urllib.parse import urlparse

    headers = {
        'Content-Type': 'application/json',
        'api-key': query_key
    }

    request_path = "/indexes/{0}/docs/suggest?{1}".format(index, params)
    
    try:
        conn = http.client.HTTPSConnection(search_host)    
        conn.request ("GET", request_path, None, headers)
        response = conn.getresponse ()
        print(response.status)
        data = response.read().decode("UTF-8")
        result = json.loads(data)
        return result

    except Exception as ex:
        raise ex

print('Ready to accept suggest queries...')

### Test 

In [ ]:
import urllib

# Request parameters
params = urllib.parse.urlencode({
    'search': 'Ma',
    'suggesterName': "location_suggester",
    'fuzzy': False,
    'highlightPreTag': '[',
    'highlightPostTag': ']',  
    'api-version':'2020-06-30'
})

result = SuggestionQuery('margies-docs-index', params)
unique = []

for val in result["value"]:    
    line = val["@search.text"].replace("\n", "")
    if (line not in unique):
        unique.append(line)
        print("\n", line)


## Autocomplete

### Query

In [ ]:
def AutoCompleteQuery (index, params):
    import http.client, json, os, sys, urllib
    from urllib.parse import urlparse

    headers = {
        'Content-Type': 'application/json',
        'api-key': query_key
    }

    request_path = "/indexes/{0}/docs/autocomplete?{1}".format(index, params)
    
    try:
        conn = http.client.HTTPSConnection(search_host)    
        conn.request ("GET", request_path, None, headers)
        response = conn.getresponse ()
        print(response.status)
        data = response.read().decode("UTF-8")
        result = json.loads(data)
        return result

    except Exception as ex:
        raise ex

print('Ready to accept autocomplete queries...')

### Test

In [ ]:
import urllib

# Request parameters
params = urllib.parse.urlencode({
    'search': 'san',
    'suggesterName': "location_suggester",
    'autocompleteMode': "twoTerms",
    'api-version':'2020-06-30'
})

result = AutoCompleteQuery('margies-docs-index', params)
unique = []

for val in result["value"]:    
    print(val["text"])

## Scoring

### Default

In [ ]:
import urllib

# Request parameters
params = urllib.parse.urlencode({
    'search': 'britain',
    'queryType': 'full',
    'searchMode': 'All',
    '$count': 'true',
    '$select':'url, file_name, size, key_phrases, last_modified',
    '$filter': "search.ismatch('reviews', 'url')",
    'api-version':'2020-06-30'
})

result = QuerySearch('margies-docs-index', params)
print("Count: " + str(result["@odata.count"]))

for val in result["value"]:    
    print("\nScore:",val["@search.score"])
    print(val["url"])
    print(val["size"], val["last_modified"])
    print(val["key_phrases"])

### Boosted

In [ ]:
import urllib

# Request parameters
params = urllib.parse.urlencode({
    'search': 'britain',
    'queryType': 'full',
    'searchMode': 'All',
    'scoringProfile': 'boostRelevance',
    '$count': 'true',
    '$select':'url, file_name, size, key_phrases, last_modified',
    '$filter': "search.ismatch('reviews', 'url')",
    'api-version':'2020-06-30'
})

result = QuerySearch('margies-docs-index', params)
print("Count: " + str(result["@odata.count"]))

for val in result["value"]:    
    print("\nScore:",val["@search.score"])
    print(val["url"])
    print(val["size"], val["last_modified"])
    print(val["key_phrases"])